# 🧠 Generador IA de bancos Moodle XML + Equilibrado y PDF

Este cuaderno:
1) **Sube un documento** base (PDF/DOCX/TXT).
2) La IA **genera preguntas MCQ** (autocontenidas, 1 correcta + 2 distractores plausibles).
3) **Equilibra** longitudes de opciones (±4 palabras) y **baraja** opciones.
4) Exporta **Moodle XML** listo para importar y un **PDF** de revisión (✅ en la correcta).

**Uso:** ejecuta las celdas en orden. Cuando pida *subir archivo*, selecciona tu documento fuente.


In [ ]:
!pip install openai==1.* beautifulsoup4 lxml reportlab pypdf python-docx tqdm --quiet

In [ ]:
# 🔑 Establece tu API key de OpenAI de forma segura (no queda guardada en el cuaderno)
import os
from getpass import getpass

if not os.getenv("OPENAI_API_KEY"):
    os.environ["OPENAI_API_KEY"] = getpass("Pega tu OPENAI_API_KEY y pulsa Enter: ")
print("✅ API key configurada en el entorno de ejecución.")

✅ API key configurada en el entorno de ejecución.


In [ ]:
# =========================
#  Núcleo: carga de documento, helpers IA, equilibrado, exportadores
# =========================
import os, json, random, re, time, traceback
from tqdm import tqdm
from pypdf import PdfReader
from docx import Document
from google.colab import files
from bs4 import BeautifulSoup
from reportlab.platypus import SimpleDocTemplate, Paragraph, Spacer
from reportlab.lib.pagesizes import A4
from reportlab.lib.styles import getSampleStyleSheet
from lxml import etree

# SDK OpenAI 1.x
from openai import OpenAI, RateLimitError

# ===== Parámetros generales =====
THRESH_DIFF = 4            # diferencia máx. de palabras entre correcta e incorrectas
RANDOM_SEED = 42           # usa None para aleatoriedad no determinista
CHUNK_MAX_CHARS = 6000     # tamaño aprox. de cada bloque del documento
OPENAI_MODEL = "gpt-4o-mini"
DEBUG_JSON = False

# Reintentos/backoff para llamadas a la IA
MAX_RETRIES = 4
BACKOFF_BASE = 2.0

if RANDOM_SEED is not None:
    random.seed(RANDOM_SEED)

# ===== Carga de documento base =====
print("📁 Sube tu documento base (PDF/DOCX/TXT)")
up = files.upload()
SRC = list(up.keys())[0]
print(f"✅ Cargado: {SRC}")

def load_text(path):
    p = path.lower()
    if p.endswith(".pdf"):
        reader = PdfReader(path)
        return "\n".join([(page.extract_text() or "") for page in reader.pages])
    if p.endswith(".docx"):
        doc = Document(path)
        return "\n".join([para.text for para in doc.paragraphs])
    with open(path, "r", encoding="utf-8", errors="ignore") as f:
        return f.read()

raw_text = load_text(SRC)
assert raw_text.strip(), "El documento parece vacío o no se pudo extraer texto."

def chunk_text(text, max_chars=10000):
    paras = [p.strip() for p in text.split("\n") if p.strip()]
    chunks, cur = [], ""
    for p in paras:
        if len(cur) + len(p) + 1 <= max_chars:
            cur += ("\n" + p) if cur else p
        else:
            chunks.append(cur); cur = p
    if cur: chunks.append(cur)
    return chunks

chunks = chunk_text(raw_text, max_chars=CHUNK_MAX_CHARS)
print(f"🧩 Bloques de texto creados: {len(chunks)} (≈{CHUNK_MAX_CHARS} chars c/u)")

# ===== Cliente OpenAI =====
api_key = os.getenv("OPENAI_API_KEY")
if not api_key:
    from getpass import getpass
    api_key = getpass("Pega tu OPENAI_API_KEY y pulsa Enter: ")
    os.environ["OPENAI_API_KEY"] = api_key
client = OpenAI(api_key=api_key)

SYSTEM_PROMPT = "Eres un generador de ítems universitarios, preciso, en español, y devuelves JSON válido."

# IMPORTANTE: llaves JSON escapadas con {{ }} para no romper str.format
USER_PROMPT_TMPL = """
Eres un generador de preguntas universitarias en psicología del lenguaje/lectura.
Crea {n} preguntas tipo test (MCQ) AUTOCONTENIDAS a partir del CONTENIDO. Nivel: universitario.
- 1 correcta + 2 distractores plausibles (sin 'todas las anteriores' / 'ninguna').
- Redacción clara, sin ambigüedad; NO dependas de "según el texto".
- Incluye justificación breve (1–2 frases) para la correcta.
- RESPONDE ÚNICAMENTE con JSON. NO incluyas explicaciones; NO uses ```json ni fences.

Estructura EXACTA:
{{
  "items": [
    {{
      "id": "BLOQUE1-Q1",
      "stem": "ENUNCIADO AUTOCONTENIDO...",
      "options": ["A...", "B...", "C..."],
      "correct_index": 1,
      "justification": "Por qué es correcta...",
      "difficulty": "media",
      "tags": ["efectos de priming","léxico"]
    }}
  ]
}}

CONTENIDO:
{content}
"""

def wc(s):
    s = re.sub(r"\s+", " ", s or "").strip()
    return len([w for w in s.split(" ") if w])

# --- Limpieza de JSON devuelto por el modelo ---
def _strip_code_fences(s: str) -> str:
    s = s.strip()
    m = re.match(r"^```(?:json)?\s*([\s\S]*?)\s*```$", s, flags=re.I)
    return m.group(1).strip() if m else s

def _extract_json_object(s: str) -> str:
    start = s.find("{")
    if start == -1: return s
    depth = 0
    for i, ch in enumerate(s[start:], start=start):
        if ch == "{": depth += 1
        elif ch == "}":
            depth -= 1
            if depth == 0: return s[start:i+1]
    return s

def _request_openai(prompt: str, retries=MAX_RETRIES):
    """Llamada con reintentos/backoff; aborta limpio si falta crédito."""
    attempt = 0
    while True:
        try:
            # Intento 1: con response_format=json_object
            return client.chat.completions.create(
                model=OPENAI_MODEL,
                messages=[{"role":"system","content":SYSTEM_PROMPT},
                          {"role":"user","content":prompt}],
                temperature=0.4,
                response_format={"type":"json_object"}
            )
        except RateLimitError as e:
            msg = str(e)
            if "insufficient_quota" in msg:
                raise RuntimeError("❌ Sin crédito en la API (insufficient_quota). Revisa plan y billing.") from e
            if attempt >= retries:
                raise
            sleep_s = BACKOFF_BASE ** attempt
            print(f"⏳ Rate limit. Reintentando en {sleep_s:.1f}s (intento {attempt+1}/{retries})…")
            time.sleep(sleep_s); attempt += 1
        except Exception as e:
            # Intento 2: sin response_format
            try:
                return client.chat.completions.create(
                    model=OPENAI_MODEL,
                    messages=[{"role":"system","content":SYSTEM_PROMPT},
                              {"role":"user","content":prompt}],
                    temperature=0.4
                )
            except Exception as e2:
                msg = str(e2)
                if "insufficient_quota" in msg:
                    raise RuntimeError("❌ Sin crédito en la API (insufficient_quota). Revisa plan y billing.") from e2
                if attempt >= retries:
                    raise
                sleep_s = BACKOFF_BASE ** attempt
                print(f"⏳ Reintentando en {sleep_s:.1f}s (intento {attempt+1}/{retries})…")
                time.sleep(sleep_s); attempt += 1

def llm_items_from_text(content, block_id="B1", n=6, debug=DEBUG_JSON):
    prompt = USER_PROMPT_TMPL.format(content=content, n=n)
    resp = _request_openai(prompt)
    raw = resp.choices[0].message.content

    # Parseo tolerante
    try:
        data = json.loads(raw)
    except Exception:
        cleaned = _strip_code_fences(raw)
        cleaned = _extract_json_object(cleaned)
        if debug:
            print("DEBUG raw[:400]:", raw[:400])
            print("DEBUG cleaned[:400]:", cleaned[:400])
        data = json.loads(cleaned)

    # Tolerancia a claves con saltos/espacios raros
    if "items" not in data:
        for k in list(data.keys()):
            if "items" in k.replace("\n","").replace(" ",""):
                data["items"] = data.pop(k)
                break

    items = data.get("items", [])
    norm = []
    for i, it in enumerate(items, start=1):
        opts = it.get("options", [])
        if len(opts) != 3:
            continue
        ci = int(it.get("correct_index", 0))
        norm.append({
            "id": it.get("id") or f"{block_id}-Q{i}",
            "stem": str(it.get("stem","")).strip(),
            "options": [str(o).strip() for o in opts],
            "correct_index": ci if 0 <= ci < 3 else 0,
            "justification": str(it.get("justification","")).strip(),
            "difficulty": it.get("difficulty","media"),
            "tags": it.get("tags",[])
        })
    return norm

def balance_and_shuffle(item, diff_threshold=THRESH_DIFF, seed=RANDOM_SEED):
    rnd = random.Random(seed)
    opts = item["options"]; ci = item["correct_index"]
    Lc = wc(opts[ci]); new_opts = opts[:]
    for i, opt in enumerate(new_opts):
        if i == ci:
            continue
        if (Lc - wc(opt)) > diff_threshold:
            extra = rnd.choice([
                " Este patrón se ha descrito en estudios de priming y decisión léxica.",
                " La literatura lo vincula con activación competitiva y control inhibitorio.",
                " Se replica en lectores con distintos niveles de proficiencia."
            ])
            new_opts[i] = (opt.strip() + extra)
    pairs = [(o, i==ci) for i,o in enumerate(new_opts)]
    rnd.shuffle(pairs)
    item["options"] = [p[0] for p in pairs]
    item["correct_index"] = next(i for i,p in enumerate(pairs) if p[1])
    return item

def validate_item(it):
    ok = True
    if len(it.get("options",[])) != 3: ok=False
    if not (0 <= it.get("correct_index", -1) < 3): ok=False
    if ok:
        s = set([o.strip().lower() for o in it["options"]])
        if len(s) < 3: ok=False
    if wc(it.get("stem","")) < 6: ok=False
    return ok

def to_moodle_xml(items, xml_path="equilibrado_IA.xml"):
    soup = BeautifulSoup('<?xml version="1.0" encoding="UTF-8"?><quiz></quiz>', "xml")
    quiz = soup.find("quiz")
    for it in items:
        q = soup.new_tag("question", type="multichoice")
        qt = soup.new_tag("questiontext", format="html")
        qt_text = soup.new_tag("text"); qt_text.string = it["stem"]
        qt.append(qt_text); q.append(qt)
        for i,opt in enumerate(it["options"]):
            ans = soup.new_tag("answer", fraction="100" if i==it["correct_index"] else "0")
            at = soup.new_tag("text"); at.string = opt
            ans.append(at); q.append(ans)
        quiz.append(q)
    xml_str = str(soup)
    root = etree.fromstring(xml_str.encode("utf-8"), parser=etree.XMLParser(recover=True))
    with open(xml_path, "wb") as f:
        f.write(etree.tostring(root, encoding="utf-8", xml_declaration=True, pretty_print=True))
    return xml_path

def to_pdf(items, pdf_path="equilibrado_IA.pdf"):
    doc = SimpleDocTemplate(pdf_path, pagesize=A4)
    styles = getSampleStyleSheet()
    story = [Paragraph("<b>Banco de preguntas (IA)</b>", styles["Title"]), Spacer(1,10)]
    for i,it in enumerate(items, start=1):
        story.append(Paragraph(f"<b>{i}. {it['stem']}</b>", styles["Normal"]))
        for j,opt in enumerate(it["options"]):
            mark = " ✅" if j==it["correct_index"] else ""
            story.append(Paragraph(f"{chr(97+j)}) {opt}{mark}", styles["Normal"]))
        if it.get("justification"):
            story.append(Paragraph(f"<i>Justificación:</i> {it['justification']}", styles["Normal"]))
        story.append(Spacer(1,8))
    doc.build(story)
    return pdf_path


In [ ]:
# =========================
#  Generación por LOTES y objetivo total (ej. 200 preguntas)
# =========================

# ⚙️ Parámetros de lote y objetivo
TARGET_ITEMS    = 200   # objetivo total del banco (ajústalo a tu necesidad)
ITEMS_PER_CALL  = 20    # cuántos ítems pide al modelo por llamada
ROUND_ROBIN     = True  # True: recorre los chunks en bucle; False: secuencial una vez
ALLOW_DUPLICATE_STEMS = False  # evitar acumular enunciados casi idénticos

# (Consejo) Si tu crédito es ajustado, baja estos números:
# TARGET_ITEMS   = 80
# ITEMS_PER_CALL = 10

assert 'chunks' in globals() and len(chunks) > 0, "No hay 'chunks' cargados. Ejecuta la celda 3 primero."
assert 'llm_items_from_text' in globals(), "Faltan helpers. Ejecuta la celda 3 primero."

collected = []
seen_stems = set()

def _norm_stem(s: str) -> str:
    return re.sub(r"\s+", " ", (s or "").strip().lower())

calls = 0
idx = 0
bar = tqdm(total=TARGET_ITEMS, desc="Acumulando ítems", unit="itm")

try:
    while len(collected) < TARGET_ITEMS:
        # Elegir chunk (round-robin o secuencial)
        if idx >= len(chunks):
            if ROUND_ROBIN:
                idx = 0
            else:
                print("⏹️ Fin de documento y objetivo no alcanzado.")
                break

        ch = chunks[idx]
        remaining = TARGET_ITEMS - len(collected)
        n_this_call = min(ITEMS_PER_CALL, remaining)

        try:
            # Generar lote
            items = llm_items_from_text(ch, block_id=f"L{calls+1}", n=n_this_call)
        except RuntimeError as e:
            # p. ej. insufficient_quota: aborta limpio y exporta lo acumulado
            print(str(e))
            break
        except Exception as e:
            print(f"⚠️ Error en llamada {calls+1} (chunk {idx+1}):", e)
            traceback.print_exc()
            idx += 1
            continue

        # Post-proceso y filtrado
        for it in items:
            it = balance_and_shuffle(it)
            if not validate_item(it):
                continue
            if not ALLOW_DUPLICATE_STEMS:
                key = _norm_stem(it.get("stem",""))
                if key in seen_stems:
                    continue
                seen_stems.add(key)

            collected.append(it)
            bar.update(1)
            if len(collected) >= TARGET_ITEMS:
                break

        calls += 1
        idx += 1

finally:
    bar.close()
    print(f"✅ Total acumulado: {len(collected)} ítems en {calls} llamadas")

# ✂️ Ajustar exactamente al objetivo por si nos pasamos
if len(collected) > TARGET_ITEMS:
    collected = collected[:TARGET_ITEMS]

# 🗂️ Exportar banco único con timestamp
from datetime import datetime
stamp = datetime.now().strftime("%Y%m%d-%H%M%S")
xml_name = f"banco_IA_{len(collected)}_{stamp}.xml"
pdf_name = f"banco_IA_{len(collected)}_{stamp}.pdf"

xml_path = to_moodle_xml(collected, xml_path=xml_name)
pdf_path = to_pdf(collected, pdf_path=pdf_name)
print("📦 XML:", xml_path)
print("📄 PDF:", pdf_path)

# Descargas
files.download(xml_path)
files.download(pdf_path)
